# Tools  🔨🪚🔩⚙️⚒️🧲🧰🔧🪛🔩⚙️🔦🧭🔗⛓️🧮
# 도구  🔨🪚🔩⚙️⚒️🧲🧰🔧🪛🔩⚙️🔦🧭🔗⛓️🧮
Tools allow agents to 'Act' in the real world. 
Careful descriptions can help your agent discover how to use your tools.

도구는 에이전트가 실제 세계에서 '행동'할 수 있게 해줍니다.
신중하게 작성된 설명은 에이전트가 도구를 어떻게 사용해야 하는지 파악하는 데 도움이 됩니다.

LangChain supports many tool formats and tool sets. Here we will cover some common cases, but check the [docs](https://docs.langchain.com/oss/python/langchain/tools) for more information.

LangChain은 다양한 도구 형식과 도구 세트를 지원합니다. 여기서는 일반적인 경우를 다루지만, 더 많은 정보는 [문서](https://docs.langchain.com/oss/python/langchain/tools)를 확인하세요.

## Setup
## 설정

Load and/or check for needed environmental variables

필요한 환경 변수를 로드하거나 확인합니다

In [ ]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")

## Calculator example
## 계산기 예제

In this example, the docstring and inferred arguments and argument types are used by the LLM to detetermine when and how to call the tool.

이 예제에서 docstring과 추론된 인자 및 인자 타입은 LLM이 도구를 언제, 어떻게 호출할지 결정하는 데 사용됩니다.

In [ ]:
from typing import Literal

from langchain.tools import tool


@tool
def real_number_calculator(
    a: float, b: float, operation: Literal["add", "subtract", "multiply", "divide"]
) -> float:
    """Perform basic arithmetic operations on two real numbers."""
    print("🧮 Invoking calculator tool")
    # Perform the specified operation
    if operation == "add":
        return a + b
    elif operation == "subtract":
        return a - b
    elif operation == "multiply":
        return a * b
    elif operation == "divide":
        if b == 0:
            raise ValueError("Division by zero is not allowed.")
        return a / b
    else:
        raise ValueError(f"Invalid operation: {operation}")

In [ ]:
from langchain.agents import create_agent

agent = create_agent(
    model="openai:gpt-5-nano",
    tools=[real_number_calculator],
    system_prompt="You are a helpful assistant",
)

This invokes your calculator tool.

이것은 계산기 도구를 호출합니다.

In [ ]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "what is 3.1125 * 4.1234"}]}
)
print(result["messages"][-1].content)

We can check the **metadata** in [LangSmith Observability](https://smith.langchain.com/public/b77bde6c-f0ad-4256-bfab-7d514ece3405/r) to see this.

[LangSmith Observability](https://smith.langchain.com/public/b77bde6c-f0ad-4256-bfab-7d514ece3405/r)에서 **메타데이터**를 확인할 수 있습니다.

The tool description can have a big impact. 
This may not  invoke your calculator tool because the inputs are integers.  (results vary from run to run)

도구 설명은 큰 영향을 미칠 수 있습니다.
입력이 정수이기 때문에 계산기 도구가 호출되지 않을 수 있습니다. (실행마다 결과가 다를 수 있음)

In [ ]:
result = agent.invoke({"messages": [{"role": "user", "content": "what is 3 * 4"}]})
print(result["messages"][-1].content)

This often does not invoke the tool though the input are real numbers. (results vary from run to run)

입력이 실수임에도 도구가 호출되지 않는 경우가 많습니다. (실행마다 결과가 다를 수 있음)

In [ ]:
result = agent.invoke({"messages": [{"role": "user", "content": "what is 3.0 * 4.0"}]})
print(result["messages"][-1].content)

## Adding a more detailed description
## 더 상세한 설명 추가하기
While a basic description is often sufficient, LangChain has support for enhanced descriptions. The example below uses one method: Google Style argument descriptions. Used with `parse_docstring=True`, this will parse and pass the arg descriptions to the model. You can rename the tool and change its description. This can be effective when you are sharing a standard tool but would like agent-specific instructions.

기본 설명만으로 충분한 경우가 많지만, LangChain은 향상된 설명을 지원합니다. 아래 예제는 Google 스타일 인자 설명을 사용하는 방법입니다. `parse_docstring=True`와 함께 사용하면 인자 설명을 파싱하여 모델에 전달합니다. 도구 이름을 변경하고 설명을 수정할 수 있습니다. 표준 도구를 공유하면서 에이전트별 지침을 추가하고 싶을 때 효과적입니다.

In [ ]:
from typing import Literal

from langchain.tools import tool


@tool(
    "calculator",
    parse_docstring=True,
    description=(
        "Perform basic arithmetic operations on two real numbers."
        "Use this whenever you have operations on any numbers, even if they are integers."
    ),
)
def real_number_calculator(
    a: float, b: float, operation: Literal["add", "subtract", "multiply", "divide"]
) -> float:
    """Perform basic arithmetic operations on two real numbers.

    Args:
        a (float): The first number.
        b (float): The second number.
        operation (Literal["add", "subtract", "multiply", "divide"]):
            The arithmetic operation to perform.

            - `"add"`: Returns the sum of `a` and `b`.
            - `"subtract"`: Returns the result of `a - b`.
            - `"multiply"`: Returns the product of `a` and `b`.
            - `"divide"`: Returns the result of `a / b`. Raises an error if `b` is zero.

    Returns:
        float: The numerical result of the specified operation.

    Raises:
        ValueError: If an invalid operation is provided or division by zero is attempted.
    """
    print("🧮  Invoking calculator tool")
    # Perform the specified operation
    if operation == "add":
        return a + b
    elif operation == "subtract":
        return a - b
    elif operation == "multiply":
        return a * b
    elif operation == "divide":
        if b == 0:
            raise ValueError("Division by zero is not allowed.")
        return a / b
    else:
        raise ValueError(f"Invalid operation: {operation}")

In [ ]:
from langchain.agents import create_agent

agent = create_agent(
    model="openai:gpt-5-nano",
    tools=[real_number_calculator],
    system_prompt="You are a helpful assistant",
)

In [ ]:
result = agent.invoke({"messages": [{"role": "user", "content": "what is 3.0 * 4.0"}]})
print(result["messages"][-1].content)

Let's check our [LangSmith Observability trace](https://smith.langchain.com/public/7d65902c-bd3c-4fc6-bbd3-7c1d66566fda/r) to see the tool description.

[LangSmith Observability 트레이스](https://smith.langchain.com/public/7d65902c-bd3c-4fc6-bbd3-7c1d66566fda/r)에서 도구 설명을 확인해 봅시다.

In [ ]:
result = agent.invoke({"messages": [{"role": "user", "content": "what is 3 * 4"}]})
print(result["messages"][-1].content)

## Try your own.
## 직접 만들어 보세요.
Create a tool of your own and try it here!

자신만의 도구를 만들어서 여기서 시도해 보세요!

In [ ]:

@tool
def your_tool(
    a: float, b: float, 
) -> float:
    """Perform your favorite operation

    Args:
        a (float): operator a description
        b (float): operator b description

    Returns:
        float: description
    """
    pass

In [ ]:
from langchain.agents import create_agent

agent = create_agent(
    model="openai:gpt-5-nano",
    tools=[your_tool],
    system_prompt="You are a helpful assistant",
)